In [14]:
#Importing all the libraries
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [15]:
import os
print(os.getcwd()+"/twitter_x_y_train.csv")

C:\Users\Kaushik\Desktop\My_Projects\Python\Sentiment Analysis/twitter_x_y_train.csv


In [16]:
#Importing dataset from the computer
training_data = pd.read_csv(os.getcwd()+"/twitter_x_y_train.csv")
#training_data = pd.read_csv("/Users/Kaushik/Desktop/Sentiment Analysis/Twitter-Sentiment-Analysis-using-NLP-master/Twitter-Sentiment-Analysis-using-NLP-master/twitter_x_y_train.csv")

In [17]:
training_data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [18]:
#Keeping the important stuff
training_data = training_data[["airline_sentiment", "text"]]

In [19]:
data_train = np.array(training_data.text)
target = np.array(training_data.airline_sentiment)

In [20]:
print(data_train.shape)
print(target.shape)


(10980,)
(10980,)


In [21]:
#Tokenizing the dataset
tok_train = list()
for i in range(len(data_train)):
    tok_train.append(word_tokenize(data_train[i]))

In [22]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [23]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    

In [24]:
from nltk import pos_tag

In [25]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
import string
punctuations = list(string.punctuation)
stop = stop + punctuations

In [26]:
def clean_review(words):
    '''
    This function takes the tokens as inputs, removes the stop words,
    and returns the lemmentized tokens
    '''
    output_words = []
    for w in words:
        if w.lower() not in stop:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [27]:
#This statement calls the clean_review function
tok_train = [clean_review(i) for i in tok_train]

In [28]:
#To change tokens back to tweets or document
X = list()
for doc in tok_train:
    X.append(" ".join(doc))

In [29]:
print(len(X))
print(len(target))

10980
10980


In [30]:
#To vectorize our dataset of words
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
count_vec = CountVectorizer(max_features = 2000, ngram_range = (1,2), max_df = 0.8)
x_features = count_vec.fit_transform(X)
x_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [32]:
#Split out dataset into test and train
x_train,x_test,y_train,y_test = train_test_split(x_features,target)

In [33]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='liblinear')
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
accuracy_score(y_test,y_pred)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7777777777777778

In [34]:
#Using Support Vector Classifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [35]:
clf = SVC()

In [36]:
#Determining the best parameter values of SVC
grid = {'C' : [1e2, 1e3, 5e3, 1e4, 5e4, 1e5], 'gamma' : [1e-3, 5e-3, 1e-4, 5e-4]}
abc =  GridSearchCV(clf, grid)
abc.fit(x_train,y_train)
abc.best_estimator_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [37]:
clf = SVC(C = 1000, gamma = 0.0001)
clf.fit(x_train,y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [38]:
y_svm = clf.predict(x_test)

In [39]:
accuracy_score(y_test,y_svm)

0.7683060109289618

In [40]:
#Using random forest classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [41]:
y_rfc = rfc.predict(x_test)

In [42]:
accuracy_score(y_rfc,y_test)

0.7304189435336976